Kaggle URL: [YoloWorld Distillation Dataset Creation](https://www.kaggle.com/code/windstorm1412/yoloworld-distillation-dataset-creation)

In [ ]:
!pip -q install ultralytics==8.3.27 open_clip_torch==2.24.0
!pip uninstall -y ray ray[default] ray[tune] >/dev/null 2>&1 || true
!pip install -q "numpy<2.0"
!pip install -q protobuf==3.20.3 tensorboard==2.14.0
!pip install -q filterpy

In [ ]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git

In [ ]:
%cd /kaggle/working/GroundingDINO/
!pip install -e .

In [ ]:
!mkdir weights
%cd weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import os, json
from transformers import BlipProcessor, BlipForConditionalGeneration
import requests
import math
import torch
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%cd /kaggle/working/GroundingDINO
from groundingdino.util.inference import load_model, load_image, predict, annotate
import warnings
warnings.filterwarnings("ignore")

dino_model = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "weights/groundingdino_swint_ogc.pth")

In [ ]:
! pip install clip

# Stage 1

In [ ]:
# =============================================================================
# CELL 1: INSTALLATION
# =============================================================================
# Chạy cell này, sau đó RESTART KERNEL trước khi chạy tiếp

!pip install -q opencv-python matplotlib
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt

In [ ]:
import sys
import os

# CRITICAL: Set environment variables FIRST, before any imports
os.environ['TORCH_JIT'] = '0'
os.environ['PYTORCH_JIT'] = '0'

print("Setting environment variables...")
print(f"TORCH_JIT = {os.environ.get('TORCH_JIT')}")
print(f"PYTORCH_JIT = {os.environ.get('PYTORCH_JIT')}")

# Patch SAM2 transforms.py to remove torch.jit.script
def patch_sam2_transforms():
    """Find and patch SAM2 transforms.py file"""
    for path in sys.path:
        transforms_file = os.path.join(path, 'sam2', 'utils', 'transforms.py')
        if os.path.exists(transforms_file):
            print(f"\n📁 Found transforms.py: {transforms_file}")
            
            # Read original content
            with open(transforms_file, 'r') as f:
                content = f.read()
            
            # Create backup
            backup_file = transforms_file + '.backup'
            if not os.path.exists(backup_file):
                with open(backup_file, 'w') as f:
                    f.write(content)
                print(f"💾 Backup created: {backup_file}")
            
            # Check if already patched
            if 'torch.jit.script(' in content:
                # Patch: Remove torch.jit.script wrapper
                original_line = 'self.transforms = torch.jit.script('
                patched_line = 'self.transforms = ('
                content = content.replace(original_line, patched_line)
                
                # Write patched version
                with open(transforms_file, 'w') as f:
                    f.write(content)
                
                print("✅ SAM2 transforms.py patched successfully!")
                print("   Removed: torch.jit.script() wrapper")
                return True
            else:
                print("ℹ️  File already patched or pattern not found")
                return True
    
    print("❌ Could not find sam2/utils/transforms.py in sys.path")
    return False

# Run patch
patch_success = patch_sam2_transforms()

if patch_success:
    print("\n" + "=" * 80)
    print("✅ Patching completed! Safe to import SAM2 now.")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print("⚠️  Patching failed. Consider using SAM1 instead.")
    print("=" * 80)


# =============================================================================
# CELL 3: IMPORT LIBRARIES
# =============================================================================
import cv2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

# NOW it's safe to import SAM2 (after patching)
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

print("=" * 80)
print("✅ All libraries imported successfully!")
print(f"📊 CUDA available: {torch.cuda.is_available()}")
print(f"🔧 TORCH_JIT: {os.environ.get('TORCH_JIT', 'not set')}")
print("=" * 80)


# =============================================================================
# CELL 4: STAGE 1 - EXTRACT LARGEST OBJECT WITH SAM2
# =============================================================================

# Configuration
SAM2_CONFIG = "sam2_hiera_l.yaml"
SAM2_CHECKPOINT = "sam2_hiera_large.pt"

def extract_largest_object_sam2_simple(
    image_path,
    model_cfg=SAM2_CONFIG,
    checkpoint_path=SAM2_CHECKPOINT,
    show=False
):
    """
    Extract largest object using SAM2 with point prompts.
    More stable than automatic mask generation.
    
    Args:
        image_path (str): Path to input image
        model_cfg (str): SAM2 config file
        checkpoint_path (str): SAM2 checkpoint path
        show (bool): Display visualization
    
    Returns:
        PIL.Image: Cropped object image
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Build SAM2 model
    sam2_model = build_sam2(model_cfg, checkpoint_path, device=device)
    predictor = SAM2ImagePredictor(sam2_model)
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {image_path}")
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    # Set image for predictor
    predictor.set_image(img_rgb)
    
    # Create point prompts: center + 4 corners
    input_points = np.array([
        [w//2, h//2],      # center
        [w//4, h//4],      # top-left region
        [3*w//4, h//4],    # top-right region
        [w//4, 3*h//4],    # bottom-left region
        [3*w//4, 3*h//4],  # bottom-right region
    ])
    input_labels = np.ones(len(input_points), dtype=np.int32)  # all foreground
    
    # Predict masks
    masks, scores, _ = predictor.predict(
        point_coords=input_points,
        point_labels=input_labels,
        multimask_output=True
    )
    
    # Handle no detection case
    if len(masks) == 0:
        print("⚠️ No object detected — returning original image")
        return Image.fromarray(img_rgb)
    
    # Select mask with highest confidence score
    best_idx = np.argmax(scores)
    best_mask = masks[best_idx]
    
    # Find bounding box of the mask
    coords = np.argwhere(best_mask)
    if len(coords) == 0:
        print("⚠️ Empty mask — returning original image")
        return Image.fromarray(img_rgb)
    
    y1, x1 = coords.min(axis=0)
    y2, x2 = coords.max(axis=0)
    
    # Crop image with mask applied
    object_crop = img_rgb[y1:y2+1, x1:x2+1]
    mask_crop = best_mask[y1:y2+1, x1:x2+1]
    object_crop = object_crop * mask_crop[:, :, np.newaxis]
    
    # Visualization
    if show:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(img_rgb)
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        axes[1].imshow(img_rgb)
        axes[1].imshow(best_mask, alpha=0.5, cmap='jet')
        axes[1].set_title(f"Best Mask (score={scores[best_idx]:.3f})")
        axes[1].axis('off')
        
        axes[2].imshow(object_crop)
        axes[2].set_title("Extracted Object")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return Image.fromarray(object_crop.astype(np.uint8))


def extract_largest_object_sam2_auto(
    image_path,
    model_cfg=SAM2_CONFIG,
    checkpoint_path=SAM2_CHECKPOINT,
    show=False
):
    """
    Extract largest object using SAM2 automatic mask generation.
    Slower but more accurate for complex scenes.
    
    Args:
        image_path (str): Path to input image
        model_cfg (str): SAM2 config file
        checkpoint_path (str): SAM2 checkpoint path
        show (bool): Display visualization
    
    Returns:
        PIL.Image: Cropped object image
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Build SAM2 model
    sam2_model = build_sam2(model_cfg, checkpoint_path, device=device)
    mask_generator = SAM2AutomaticMaskGenerator(sam2_model)
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {image_path}")
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Generate all masks automatically
    masks = mask_generator.generate(img_rgb)
    
    if len(masks) == 0:
        print("⚠️ No object detected — returning original image")
        return Image.fromarray(img_rgb)
    
    # Find largest mask by area
    largest_mask = max(masks, key=lambda x: x['area'])
    segmentation = largest_mask['segmentation']
    bbox = largest_mask['bbox']  # [x, y, w, h]
    
    x, y, w, h = map(int, bbox)
    
    # Crop with mask
    object_crop = img_rgb[y:y+h, x:x+w]
    mask_crop = segmentation[y:y+h, x:x+w]
    object_crop = object_crop * mask_crop[:, :, np.newaxis]
    
    # Visualization
    if show:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(img_rgb)
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        axes[1].imshow(img_rgb)
        axes[1].imshow(segmentation, alpha=0.5, cmap='jet')
        axes[1].set_title(f"Largest Mask (area={largest_mask['area']})")
        axes[1].axis('off')
        
        axes[2].imshow(object_crop)
        axes[2].set_title("Extracted Object")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return Image.fromarray(object_crop.astype(np.uint8))

# Stage 2

In [ ]:
## ----------------------STAGE 2-----------------------------------
def extract_main_object_caption(image_path, model_id="Salesforce/blip-image-captioning-base", prompt="The single most important object in this image is a"):
    """
    Xác định vật thể chính trong ảnh bằng mô hình BLIP image captioning.

    Args:
        image_path (str): Đường dẫn ảnh đầu vào.
        model_id (str): ID mô hình BLIP từ Hugging Face Hub.
        prompt (str): Prompt mô tả vật thể cần tìm trong ảnh.

    Returns:
        str: Tên vật thể chính được BLIP mô tả.
    """

    # Load model và processor
    processor = BlipProcessor.from_pretrained(model_id)
    blip_model = BlipForConditionalGeneration.from_pretrained(model_id)

    # Mở ảnh
    image = Image.open(image_path).convert("RGB")

    # Chuẩn bị input
    prompt = "The single most important object in this image is a" 
    inputs = processor(images=image, text=prompt, return_tensors="pt")

    # Sinh mô tả ngắn
    out = blip_model.generate(**inputs, max_new_tokens=5)

    # Giải mã output
    generated_text = processor.decode(out[0], skip_special_tokens=True)

    # Trích xuất vật thể
    reference_object = (
        generated_text.replace(prompt.lower(), "")
        .replace(prompt, "")
        .strip()
        .split('.')[0]
        .strip()
    )
    return reference_object


# Stage 3

In [ ]:
def annotate_on_frame(image_path, text_prompt, box_threshold=0.35, text_threshold=0.25, max_crop_ratio=0.7):
    """
    Annotates an input image with detected objects và các phrases từ GroundingDINO.
    Giữ nguyên màu ảnh gốc.

    Parameters:
    - max_crop_ratio: float, crop chiếm tỉ lệ > max_crop_ratio so với frame sẽ bị loại bỏ

    Returns:
    - annotated_frame (numpy.ndarray)
    - abs_box (list of [x0,y0,x1,y1])
    - logits
    - phrases
    - crops (list of numpy.ndarray)
    """
    import torch
    import numpy as np
    import cv2

    image_source, image = load_image(image_path)
    ht, wd = image_source.shape[:2]

    boxes, logits, phrases = predict(
        model=dino_model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )

    abs_box = []
    crops = []

    for box, phrase in zip(boxes, phrases):
        cx, cy, bw, bh = box.detach().cpu().numpy()
        x0 = int(round((cx - bw/2) * wd))
        y0 = int(round((cy - bh/2) * ht))
        x1 = int(round((cx + bw/2) * wd))
        y1 = int(round((cy + bh/2) * ht))

        # Clip vào ảnh
        x0 = max(0, min(x0, wd-1))
        y0 = max(0, min(y0, ht-1))
        x1 = max(0, min(x1, wd-1))
        y1 = max(0, min(y1, ht-1))

        # Loại bỏ crop quá lớn
        crop_area = (x1 - x0) * (y1 - y0)
        frame_area = ht * wd
        if crop_area / frame_area > max_crop_ratio:
            continue  # bỏ crop này

        abs_box.append([x0, y0, x1, y1])
        crop = image_source[y0:y1, x0:x1].copy()
        crops.append(crop)

    # Vẽ bounding box lên ảnh gốc
    annotated_frame = image_source.copy()
    for (x0, y0, x1, y1), phrase in zip(abs_box, phrases):
        cv2.rectangle(annotated_frame, (x0, y0), (x1, y1), (255,0,0), 2)
        cv2.putText(annotated_frame, phrase, (x0, max(y0-5,0)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

    return annotated_frame, abs_box, logits, phrases, crops


# Stage 4

In [ ]:
## ----------------------STAGE 4-----------------------------------
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
import cv2

# ---------------- Load CLIP model once ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 0.508
# clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch16").to(device)
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch16")

#0.3522
# clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K").to(device)
# processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

# clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K").to(device)
# processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

def compare_crops_with_reference(reference_img_array, crops):
    """
    So sánh nhiều crop với 1 reference image bằng CLIP (không hiển thị).

    Parameters:
    - reference_img_array: numpy array (H,W,3), BGR hoặc RGB
    - crops: list of numpy arrays (H_crop,W_crop,3)

    Returns:
    - similarities: list các giá trị similarity (rỗng nếu không có crop)
    - best_idx: index của crop match tốt nhất (None nếu không có crop)
    - best_similarity: similarity cao nhất (None nếu không có crop)
    """
    if len(crops) == 0:
        return [], None, None

    # Chuyển reference image sang RGB nếu cần
    if reference_img_array.shape[2] == 3:
        reference_rgb = cv2.cvtColor(reference_img_array, cv2.COLOR_BGR2RGB) \
            if reference_img_array.dtype != np.float32 else reference_img_array
    else:
        reference_rgb = reference_img_array

    reference_img = Image.fromarray(reference_rgb)
    ref_inputs = processor(images=reference_img, return_tensors="pt").to(device)

    with torch.no_grad():
        ref_features = clip_model.get_image_features(**ref_inputs)
        ref_features = ref_features / ref_features.norm(dim=-1, keepdim=True)

    similarities = []

    for crop in crops:
        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB) if crop.shape[2] == 3 else crop
        crop_pil = Image.fromarray(crop_rgb)
        crop_inputs = processor(images=crop_pil, return_tensors="pt").to(device)

        with torch.no_grad():
            crop_features = clip_model.get_image_features(**crop_inputs)
            crop_features = crop_features / crop_features.norm(dim=-1, keepdim=True)

        # Cosine similarity
        sim = (ref_features @ crop_features.T).item()
        similarities.append(sim)
    if len(similarities) == 0:
        return [], None, None

    best_idx = int(np.argmax(similarities))
    best_similarity = similarities[best_idx]

    return similarities, best_idx, best_similarity

In [ ]:
def get_label_from_folder(folder_name):
    # Loại bỏ phần số và gạch dưới ở cuối (vd: Backpack_0 -> Backpack)
    label = re.sub(r'_\d+$', '', folder_name)
    # Thay thế gạch dưới giữa các từ bằng khoảng trắng (nếu có)
    return label.replace('_', ' ')

In [ ]:
import os
import cv2
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import yaml
import re
import gc

# --- Cấu hình ---
DATA_SOURCES = ["/kaggle/input/train-zaic-dl", "/kaggle/input/zaic-test-frames"]
DISTILL_DIR = "/kaggle/working/yolo_world_distill"
IMG_PATH = os.path.join(DISTILL_DIR, "images")
LBL_PATH = os.path.join(DISTILL_DIR, "labels")
os.makedirs(IMG_PATH, exist_ok=True)
os.makedirs(LBL_PATH, exist_ok=True)

# 1. Tạo bộ từ điển chuẩn từ tên các folder dữ liệu
unique_labels = set()
for src in DATA_SOURCES:
    if os.path.exists(src):
        for folder in os.listdir(src):
            if os.path.isdir(os.path.join(src, folder)):
                unique_labels.add(get_label_from_folder(folder))

class_list = sorted(list(unique_labels)) 
label_map = {name: i for i, name in enumerate(class_list)}
SIMILARITY_THRESHOLD = 0.65 

# 2. Vòng lặp gán nhãn
for root_path in DATA_SOURCES:
    if not os.path.exists(root_path): continue
    
    for folder_name in sorted(os.listdir(root_path)):
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.isdir(folder_path): continue
        
        # Nhãn mục tiêu cố định cho folder này (Để Student học đồng nhất)
        target_label = get_label_from_folder(folder_name)
        class_id = label_map[target_label]

        # A. TEACHER PREPARATION (Dùng BLIP để lấy prompt mô tả cho DINO)
        obj_images_dir = os.path.join(folder_path, "object_images")
        ref_img_path = os.path.join(obj_images_dir, "img_1.jpg")
        
        ref_img_pil = extract_largest_object_sam2_simple(ref_img_path)
        # Prompt này chỉ dùng để DINO "biết đường mà tìm", không dùng làm nhãn lưu trữ
        dino_prompt = extract_main_object_caption(ref_img_path)

        gc.collect()
        torch.cuda.empty_cache() #
        
        # B. AUTO-LABELING
        frames_path = os.path.join(folder_path, "object_frames")
        for frame_name in tqdm(os.listdir(frames_path), desc=f"Distilling {folder_name}"):
            frame_p = os.path.join(frames_path, frame_name)
            
            # GroundingDINO tìm các vùng khả nghi theo mô tả của BLIP
            _, abs_boxes, _, _, crops = annotate_on_frame(frame_p, dino_prompt)
            
            # CLIP lọc lại vùng giống nhất với ảnh mẫu (Few-shot logic)
            sims, best_idx, best_sim = compare_crops_with_reference(np.array(ref_img_pil), crops)
            
            if best_idx is not None and best_sim >= SIMILARITY_THRESHOLD:
                new_img_name = f"{folder_name}_{frame_name}"
                frame_img = cv2.imread(frame_p)
                
                # Lưu nhãn với class_id đã được định nghĩa từ folder
                h, w = frame_img.shape[:2]
                box = abs_boxes[best_idx]
                
                # Viết file nhãn YOLO
                with open(os.path.join(LBL_PATH, new_img_name.replace(".jpg", ".txt")), "w") as f:
                    cx, cy = (box[0] + box[2])/2/w, (box[1] + box[3])/2/h
                    bw, bh = (box[2] - box[0])/w, (box[3] - box[1])/h
                    f.write(f"{class_id} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")
                
                cv2.imwrite(os.path.join(IMG_PATH, new_img_name), frame_img)

# 3. Tự động tạo file cấu hình cho Student
with open('/kaggle/working/data.yaml', 'w') as f:
    yaml.dump({'train': IMG_PATH, 'val': IMG_PATH, 'nc': len(class_list), 'names': class_list}, f)